In [1]:
import sys
sys.path.append('../src')

import network as nw
import road as rd
import junction as jn
import traffic_lights as tl
import torch
import openGLUtils as glutils
import openGLUtilsGif as glgif
import matplotlib.pyplot as plt
import FV_schemes as fv
import scipy.integrate as integrate
import numpy as np
import roundabout as rb

In [2]:
L = 25 # Length of road
N = 2 # Number of nodes to be used for every 50 meters

# Vestre Strandgate roundabout:
# Four arm roundabout with 4 junctions, where 2 roads are a part of simulation
# Secondary roads can be described by a small object with a queue and an inflow condition
# Creating the mainline
main_speed_limit = torch.tensor([50.0], requires_grad=True)
vs_main_1 = rd.Road(1, L, N, main_speed_limit, [], left_pos=(0,0), right_pos=(0.5, -0.5),
                    inflow = 0.0, id="vs_mainline_1")
vs_main_2 = rd.Road(1, L, N, main_speed_limit, [], left_pos=(0.5, -0.5), right_pos=(0, -1),
                    inflow = 0.0, id="vs_mainline_2")
vs_main_3 = rd.Road(1, L, N, main_speed_limit, [], left_pos=(0, -1), right_pos=(-0.5, -0.5),
                    inflow = 0.0, id="vs_mainline_3")
vs_main_4 = rd.Road(1, L, N, main_speed_limit, [], left_pos=(-0.5, -0.5), right_pos=(0, 0),
                    inflow = 0.0, id="vs_mainline_4")

In [3]:
secondary_gamma = torch.tensor(50 / (25 * 3.6))
max_inflow = fv.flux(torch.tensor(0.5), secondary_gamma)
# Assume constant inflow from all roads
rho_1, rho_2, rho_3 = torch.tensor(0.4),torch.tensor(0.05), torch.tensor(0.05)
inflow_1 = lambda t : fv.flux(rho_1, secondary_gamma)
inflow_2 = lambda t : fv.flux(rho_2, secondary_gamma)
inflow_3 = lambda t : fv.flux(rho_3, secondary_gamma)

In [4]:
max_inflow

tensor(0.1389)

In [5]:
# Secondary roads
vs_fw = rd.Road(2, L, N, main_speed_limit, [], left_pos=(0.0,0.0), right_pos=(0.0,1),
                    inflow = 0.0, id="vs_fw")
vs_bw = rd.Road(2, L, N, main_speed_limit, [], left_pos=(0.0,1), right_pos=(0,0),
                    inflow = 0.0, id="vs_bw")
vs_secondary_1 = rb.RoundaboutRoad(inflow_1, max_inflow)
vs_secondary_2 = rb.RoundaboutRoad(inflow_2, max_inflow)
vs_secondary_3 = rb.RoundaboutRoad(inflow_3, max_inflow)
vs_jnc_1 = rb.RoundaboutJunction(vs_main_1, vs_main_2, 0.7, vs_fw, vs_bw, queue_junction = False)
vs_jnc_2 = rb.RoundaboutJunction(vs_main_2, vs_main_3, 0.7, vs_secondary_1, None, queue_junction = True)
vs_jnc_3 = rb.RoundaboutJunction(vs_main_3, vs_main_4, 0.7, vs_secondary_2, None, queue_junction = True)
vs_jnc_4 = rb.RoundaboutJunction(vs_main_4, vs_main_1, 0.7, vs_secondary_3, None, queue_junction = True)
vs_junctions = [vs_jnc_1, vs_jnc_2, vs_jnc_3, vs_jnc_4]

In [6]:
# Create the roundabout
roundabout = rb.Roundabout([vs_main_1, vs_main_2, vs_main_3, vs_main_4], 
                           [vs_fw, vs_secondary_1, vs_secondary_2, vs_secondary_3],
                           [vs_bw, None, None, None],vs_junctions)

In [7]:
# Create network:
roads = [vs_main_1, vs_main_2, vs_main_3, vs_main_4, vs_fw, vs_bw]
network = nw.RoadNetwork(roads, [], 50, [roundabout])

In [8]:
densities, queues, _, _ = network.solve_cons_law()

In [9]:
glgif.draw_timed_with_shift(network, densities, interval_seconds = 0.05, output_name = 'roundabout.gif')

: 

In [ ]:
ASFGHKJ

In [ ]:
vs_secondary_1.queue_length

tensor(0., grad_fn=<MaximumBackward0>)

In [ ]:
# vs_jnc_2:
jnc = vs_jnc_2
main_in = jnc.mainline_in
main_out = jnc.mainline_out
second_in = jnc.secondary_in
beta = rb.priority_fnc(main_in.rho[-1])

In [ ]:
densities

{0: {0: tensor([0., 0., 0., 0., 0., 0.]),
  tensor(0.6000, grad_fn=<AddBackward0>): tensor([0.0475, 0.0475, 0.0000, 0.0000, 0.0000, 0.0000], grad_fn=<CopySlices>),
  tensor(1.2000, grad_fn=<AddBackward0>): tensor([0.0495, 0.0495, 0.0234, 0.0224, 0.0221, 0.0221], grad_fn=<CopySlices>),
  tensor(1.8000, grad_fn=<AddBackward0>): tensor([0.0564, 0.0564, 0.0365, 0.0346, 0.0340, 0.0340], grad_fn=<CopySlices>),
  tensor(2.4000, grad_fn=<AddBackward0>): tensor([0.0605, 0.0605, 0.0465, 0.0445, 0.0437, 0.0437], grad_fn=<CopySlices>),
  tensor(3.0000, grad_fn=<AddBackward0>): tensor([0.0653, 0.0653, 0.0535, 0.0519, 0.0511, 0.0511], grad_fn=<CopySlices>),
  tensor(3., grad_fn=<AddBackward0>): tensor([0.0653, 0.0653, 0.0535, 0.0519, 0.0511, 0.0511], grad_fn=<CopySlices>)},
 1: {0: tensor([0., 0., 0., 0., 0., 0.]),
  tensor(0.6000, grad_fn=<AddBackward0>): tensor([0., 0., 0., 0., 0., 0.], grad_fn=<CopySlices>),
  tensor(1.2000, grad_fn=<AddBackward0>): tensor([0., 0., 0., 0., 0., 0.], grad_fn=<CopyS

In [ ]:
t = 50

In [ ]:
main_rho_in = main_in.rho[-1]
main_rho_out = main_out.rho[-1]
main_max_dens = main_in.max_dens
main_gamma = main_in.gamma[main_in.idx]

demands = torch.zeros(2)
demands[0] = main_max_dens * fv.D(main_rho_in, main_gamma)
demands[1] = second_in.demand(t)
# Demands look okay...

In [ ]:
supply = main_max_dens * fv.S(main_rho_out, main_gamma)
supply
# Supply also looks good

tensor(0.1389, grad_fn=<MulBackward0>)

In [ ]:
# alpha = 0.7
alpha = 0.7
out_flux = torch.min((1-alpha)*demands[0] + demands[1])
in_fluxes = torch.zeros(2)
max_main_in = torch.max(beta*supply, supply - demands[1])
in_fluxes[0] = 1/(1-alpha) * torch.min((1-alpha)*demands[0], max_main_in)
max_second_in = torch.max((1-beta)*supply, supply - (1-alpha)*demands[0])
in_fluxes[1] = torch.min(demands[1], max_second_in)

In [ ]:
out_flux # This is okay
in_fluxes # This is wrong!

tensor([4.9659e-05, 1.3887e-01], grad_fn=<CopySlices>)

In [ ]:
out_flux
dt = 1.2
print(main_out.rho)
main_out.update_left_boundary(out_flux, dt)
print(main_out.rho)

tensor([0.0776, 0.0776, 0.0412, 0.0412, 0.0412, 0.0412], grad_fn=<CopySlices>)
tensor([0.4332, 0.4332, 0.0412, 0.0412, 0.0412, 0.0412], grad_fn=<CopySlices>)


In [ ]:
ahfdjk
# glgif.draw_timed_with_shift(network, densities, interval_seconds = 0.05, output_name = 'roundabout.gif')

NameError: name 'ahfdjk' is not defined